In [1]:
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from typing import List
import pandas as pd
from gurobipy import GRB

In [2]:
def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

def allPathsSourceTarget(graph: List[List[int]]) -> List[List[int]]:
    stack = [[0]]
    paths = []
    maxrun = 10000
    counter = 0
#     print("paths",paths)
#     print("just in")
    while stack:
        path = stack.pop()
        node = path[-1]
#         print("Node under consideration :",node)
        if node == len(graph) - 1:
            paths.append(path)
#             print("WOHOOOO",paths)
            
            counter+=1
            print("found a path",counter," : ",path)
            if(counter == maxrun):
                return paths
            continue
            
        for new_node in graph[node]:
#             print("Nodes coming out :",new_node)
            stack.append((path + [new_node]))
                    
#     print(paths)    
    return paths

def distinctCustomer(path,nodeCount):
    customer = 0
    visited = [] # stores index of visited customer
    for i in range(1,len(path)-1):
        node = path[i]
        for j in range(0,len(nodeCount)):
            if(node in nodeCount[j]):
                if(j not in visited):
                    visited.append(j)
                    customer+=1
    return customer
def getmin(path,ans,new_edges):
    for i in range(0,len(path)-1):
        edge = (path[i],path[i+1])
        minVal = 99 # any number > 1
        for key in new_edges:
            if(new_edges[key] == edge):
                minVal = min(minVal,ans[key])
    return minVal

def mycallback(model, where):

    if where == GRB.Callback.BARRIER:
        # Barrier callback
        itcnt = model.cbGet(GRB.Callback.BARRIER_ITRCNT)
        primobj = model.cbGet(GRB.Callback.BARRIER_PRIMOBJ)
        dualobj = model.cbGet(GRB.Callback.BARRIER_DUALOBJ)
        priminf = model.cbGet(GRB.Callback.BARRIER_PRIMINF)
        dualinf = model.cbGet(GRB.Callback.BARRIER_DUALINF)
        cmpl = model.cbGet(GRB.Callback.BARRIER_COMPL)
        if abs(primobj - dualobj) <= 1:
            print("Custom gap reached")
            model.terminate()
        print("%d %g %g %g %g %g" % (itcnt, primobj, dualobj, priminf, dualinf, cmpl))


In [3]:
# timeWindows = [[170, 290], [480, 640], [130, 190], [280, 330], [450, 590], [90, 100], [260, 410], [100, 160], [140, 150], [140, 250], [110, 150], [460, 570], [200, 350], [100, 270], [500, 560], [470, 590], [120, 130], [420, 600], [500, 690], [300, 380], [140, 210], [130, 270], [120, 220], [290, 490], [340, 430], [340, 380], [240, 310], [480, 540], [220, 230], [40, 110], [290, 300], [220, 230], [500, 560], [440, 540], [370, 550], [190, 230], [500, 670], [200, 400], [170, 250], [380, 450]]

# timeMatrix = [[0, 50, 60, 30, 60, 40, 50, 100, 100, 10, 50, 70, 60, 70, 50, 10, 60, 60, 40, 50, 60, 10, 40, 40, 90, 10, 20, 40, 90, 30, 50, 70, 90, 60, 80, 50, 60, 40, 60, 20], [50, 0, 
# 60, 50, 70, 10, 50, 20, 20, 90, 70, 80, 40, 100, 70, 80, 40, 20, 40, 60, 20, 60, 70, 100, 90, 60, 10, 70, 80, 10, 20, 20, 30, 70, 70, 20, 80, 100, 10, 40], [60, 60, 0, 100, 100, 100, 
# 40, 100, 80, 10, 100, 60, 20, 80, 20, 80, 30, 90, 90, 60, 100, 40, 20, 50, 80, 60, 90, 10, 60, 20, 20, 70, 90, 10, 80, 40, 30, 20, 90, 60], [30, 50, 100, 0, 60, 70, 70, 20, 80, 40, 50, 80, 40, 60, 50, 40, 60, 10, 70, 90, 20, 70, 60, 30, 70, 20, 10, 90, 80, 100, 50, 30, 70, 80, 80, 60, 10, 60, 70, 70], [60, 70, 100, 60, 0, 30, 60, 60, 50, 70, 70, 80, 10, 30, 40, 60, 60, 60, 100, 80, 90, 60, 50, 100, 30, 10, 70, 40, 10, 90, 70, 10, 60, 80, 100, 20, 50, 50, 30, 70], [40, 10, 100, 70, 30, 0, 70, 10, 40, 30, 20, 30, 20, 90, 80, 100, 30, 50, 10, 60, 90, 40, 60, 20, 50, 20, 40, 30, 50, 70, 70, 20, 40, 70, 30, 90, 50, 30, 90, 10], [50, 50, 40, 70, 60, 70, 0, 60, 60, 100, 100, 50, 80, 100, 80, 40, 20, 10, 100, 20, 30, 10, 60, 20, 80, 10, 90, 80, 80, 50, 80, 100, 90, 70, 20, 20, 40, 80, 10, 40], [100, 20, 100, 20, 60, 10, 60, 0, 20, 80, 30, 80, 10, 90, 20, 30, 90, 90, 30, 30, 60, 100, 90, 20, 10, 10, 40, 70, 90, 20, 50, 50, 70, 100, 70, 90, 30, 10, 10, 20], [100, 20, 80, 80, 50, 40, 60, 20, 0, 80, 70, 50, 20, 100, 80, 80, 40, 70, 70, 90, 100, 90, 60, 20, 90, 60, 50, 10, 10, 60, 20, 60, 100, 
# 30, 10, 100, 10, 10, 70, 10], [10, 90, 10, 40, 70, 30, 100, 80, 80, 0, 60, 10, 30, 10, 80, 60, 50, 20, 70, 70, 80, 80, 40, 90, 20, 90, 20, 30, 60, 20, 100, 70, 70, 30, 80, 70, 10, 80, 80, 10], [50, 70, 100, 50, 70, 20, 100, 30, 70, 60, 0, 100, 80, 90, 60, 60, 10, 20, 30, 100, 90, 30, 20, 40, 90, 80, 40, 100, 50, 50, 30, 10, 70, 20, 20, 50, 10, 10, 100, 30], [70, 80, 60, 80, 80, 30, 50, 80, 50, 10, 100, 0, 80, 90, 50, 20, 20, 50, 30, 50, 70, 50, 100, 80, 10, 90, 20, 100, 70, 80, 100, 50, 40, 30, 100, 40, 100, 20, 40, 90], [60, 40, 20, 40, 10, 20, 80, 10, 20, 30, 80, 80, 0, 50, 60, 60, 50, 70, 50, 50, 10, 20, 10, 20, 100, 70, 90, 50, 70, 80, 10, 90, 30, 30, 30, 40, 70, 70, 40, 90], [70, 100, 80, 60, 30, 90, 100, 90, 100, 10, 90, 90, 50, 0, 20, 40, 30, 20, 40, 10, 80, 60, 60, 10, 10, 10, 80, 60, 60, 80, 40, 20, 60, 20, 60, 60, 40, 10, 50, 80], [50, 70, 20, 50, 40, 80, 80, 20, 80, 80, 60, 50, 60, 20, 0, 80, 50, 30, 90, 60, 10, 10, 50, 70, 50, 30, 10, 20, 100, 20, 30, 100, 70, 60, 40, 40, 30, 50, 10, 20], [10, 80, 80, 40, 60, 100, 40, 30, 80, 60, 60, 20, 60, 40, 80, 0, 90, 50, 40, 70, 90, 90, 80, 30, 10, 40, 70, 50, 40, 60, 10, 60, 10, 10, 60, 100, 20, 90, 60, 10], [60, 40, 30, 60, 60, 30, 20, 90, 40, 50, 10, 20, 50, 30, 50, 90, 0, 10, 70, 50, 50, 90, 90, 70, 40, 20, 30, 10, 20, 60, 80, 80, 90, 80, 30, 10, 60, 30, 100, 60], [60, 20, 90, 10, 60, 50, 10, 90, 70, 20, 20, 50, 70, 20, 30, 50, 10, 0, 30, 30, 70, 60, 70, 90, 50, 90, 60, 80, 20, 70, 90, 50, 60, 10, 50, 10, 50, 50, 30, 90], [40, 40, 90, 70, 100, 10, 100, 30, 70, 70, 30, 30, 50, 40, 90, 40, 70, 30, 0, 70, 50, 10, 10, 50, 90, 40, 90, 30, 30, 40, 10, 50, 60, 20, 80, 80, 20, 60, 80, 80], [50, 60, 60, 90, 80, 60, 20, 30, 90, 70, 100, 50, 50, 10, 60, 70, 50, 30, 70, 0, 40, 40, 70, 100, 70, 70, 70, 100, 100, 50, 50, 30, 90, 60, 50, 100, 100, 20, 20, 100], [60, 20, 100, 20, 90, 90, 30, 60, 100, 80, 90, 70, 10, 80, 10, 90, 50, 70, 50, 40, 0, 100, 20, 100, 40, 100, 50, 40, 80, 80, 30, 50, 90, 50, 70, 50, 20, 60, 10, 60], [10, 60, 40, 70, 60, 40, 10, 100, 90, 80, 30, 50, 20, 60, 10, 90, 90, 60, 10, 40, 100, 0, 60, 90, 90, 30, 40, 70, 60, 50, 90, 80, 80, 40, 70, 90, 70, 50, 70, 10], [40, 70, 20, 60, 50, 60, 60, 90, 60, 40, 20, 100, 10, 60, 50, 80, 90, 70, 10, 70, 20, 60, 0, 100, 40, 80, 10, 10, 60, 10, 20, 40, 40, 30, 10, 70, 50, 60, 50, 80], [40, 100, 50, 30, 100, 20, 20, 20, 20, 90, 40, 80, 20, 10, 70, 30, 70, 90, 50, 100, 100, 90, 100, 0, 20, 70, 70, 40, 20, 60, 60, 10, 60, 60, 60, 90, 100, 100, 80, 30], [90, 90, 80, 70, 30, 50, 80, 10, 90, 20, 90, 10, 100, 10, 50, 10, 40, 50, 90, 70, 40, 90, 40, 20, 0, 80, 20, 100, 50, 20, 70, 90, 20, 30, 100, 50, 20, 40, 90, 70], [10, 60, 60, 20, 10, 20, 10, 10, 60, 90, 80, 90, 70, 10, 30, 40, 20, 90, 40, 70, 100, 30, 80, 70, 80, 0, 60, 90, 100, 80, 60, 10, 80, 20, 50, 40, 90, 10, 90, 90], [20, 10, 90, 10, 70, 40, 90, 40, 50, 20, 40, 20, 90, 80, 10, 70, 30, 60, 90, 70, 50, 40, 10, 70, 20, 60, 0, 
# 60, 80, 20, 100, 90, 40, 20, 90, 30, 100, 40, 80, 40], [40, 70, 10, 90, 40, 30, 80, 70, 10, 30, 100, 100, 50, 60, 20, 50, 10, 80, 30, 100, 40, 70, 10, 40, 100, 90, 60, 0, 10, 90, 60, 
# 100, 30, 100, 60, 70, 90, 100, 40, 20], [90, 80, 60, 80, 10, 50, 80, 90, 10, 60, 50, 70, 70, 60, 100, 40, 20, 20, 30, 100, 80, 60, 60, 20, 50, 100, 80, 10, 0, 30, 20, 40, 90, 80, 60, 
# 40, 10, 60, 100, 50], [30, 10, 20, 100, 90, 70, 50, 20, 60, 20, 50, 80, 80, 80, 20, 60, 60, 70, 40, 50, 80, 50, 10, 60, 20, 80, 20, 90, 30, 0, 40, 10, 30, 90, 60, 50, 40, 10, 20, 50], [50, 20, 20, 50, 70, 70, 80, 50, 20, 100, 30, 100, 10, 40, 30, 10, 80, 90, 10, 50, 30, 90, 20, 60, 70, 60, 100, 60, 20, 40, 0, 100, 70, 60, 70, 30, 70, 10, 40, 30], [70, 20, 70, 30, 
# 10, 20, 100, 50, 60, 70, 10, 50, 90, 20, 100, 60, 80, 50, 50, 30, 50, 80, 40, 10, 90, 10, 90, 100, 40, 10, 100, 0, 70, 30, 20, 40, 90, 80, 90, 50], [90, 30, 90, 70, 60, 40, 90, 70, 100, 70, 70, 40, 30, 60, 70, 10, 90, 60, 60, 90, 90, 80, 40, 60, 20, 80, 40, 30, 90, 30, 70, 70, 0, 40, 90, 70, 100, 70, 70, 80], [60, 70, 10, 80, 80, 70, 70, 100, 30, 30, 20, 30, 30, 20, 60, 10, 80, 10, 20, 60, 50, 40, 30, 60, 30, 20, 20, 100, 80, 90, 60, 30, 40, 0, 30, 60, 60, 80, 100, 20], [80, 70, 80, 80, 100, 30, 20, 70, 10, 80, 20, 100, 30, 60, 40, 60, 30, 50, 80, 50, 70, 70, 10, 60, 100, 50, 90, 60, 60, 60, 70, 20, 90, 30, 0, 90, 20, 30, 50, 40], [50, 20, 40, 60, 20, 90, 20, 90, 100, 70, 50, 40, 40, 60, 40, 100, 10, 10, 80, 100, 50, 90, 70, 90, 50, 40, 30, 70, 40, 50, 30, 40, 70, 60, 90, 0, 60, 70, 80, 20], [60, 80, 30, 10, 50, 50, 40, 30, 10, 10, 10, 100, 70, 40, 30, 20, 60, 50, 20, 100, 20, 70, 50, 100, 20, 90, 100, 90, 10, 40, 70, 90, 100, 60, 20, 60, 0, 100, 30, 100], [40, 100, 20, 60, 50, 30, 80, 10, 10, 80, 10, 20, 70, 10, 50, 90, 30, 50, 60, 20, 60, 50, 60, 100, 40, 10, 40, 100, 60, 10, 10, 80, 70, 80, 30, 70, 100, 0, 30, 70], [60, 10, 90, 70, 30, 90, 10, 10, 70, 80, 100, 40, 40, 50, 10, 60, 100, 30, 80, 20, 10, 70, 50, 80, 90, 90, 80, 40, 100, 20, 40, 90, 70, 100, 50, 
# 80, 30, 30, 0, 90], [20, 40, 60, 70, 70, 10, 40, 20, 10, 10, 30, 90, 90, 80, 20, 10, 60, 90, 80, 100, 60, 10, 80, 30, 70, 90, 40, 20, 50, 50, 30, 50, 80, 20, 40, 20, 100, 70, 90, 0]] 

# # discreteTimeWindows = []
# # for i in timeWindows:
# #     discreteTimeWindows.append(discreteTime(i, 10))


# print(len(timeWindows))
# for i in timeMatrix:
#     if(len(i)!=len(timeWindows)):
#         print("F")
# print('T')

In [4]:


# # # 20 customer -> 1.1 minute
# # # 30 customers -> 5 minute    1.69 minutes
# # # 40                          2.45 min
# # # 50 customer -> 160 minute







# # # customers = 2

# # timeWindows = [[20, 30], [20, 30], [10, 20], [20, 40], [20, 40], [20, 50], [10, 30], [10, 30], [20, 30], [20, 30]]
# # discreteTimeWindows = [[20, 30], [20, 30], [10, 20], [20, 30, 40], [20, 30, 40], [20, 30, 40, 50], [10, 20, 30], [10, 20, 30], [20, 30], [20, 30]]
# # timeMatrix = [[0, 40, 70, 20, 90, 90, 60, 80, 40, 30], [40, 0, 10, 40, 40, 70, 50, 30, 30, 60], [70, 10, 0, 80, 100, 60, 50, 50, 70, 100], [20, 40, 80, 0, 80, 70, 40, 10, 70, 80], [90, 40, 100, 80, 0, 100, 10, 100, 40, 40], [90, 70, 60, 70, 100, 0, 10, 20, 20, 90], [60, 50, 50, 40, 10, 10, 0, 40, 80, 10], [80, 30, 50, 10, 100, 20, 40, 0, 100, 80], [40, 30, 70, 70, 40, 20, 80, 100, 0, 60], [30, 60, 100, 80, 40, 90, 10, 80, 60, 0]]
# # customers = len(timeWindows)
# # ## removing unreachable from source  
# # startingTime = 0
# # ## if customers are unreacble then startingTime + 






# # # 
# customers = 40
# timeWindows = []
# for i in range(0,customers):
#     st = random.randint(1,2)*10
#     et = random.randint(1,3)*10 + st
#     tw = [st,et]
#     timeWindows.append(tw)

# print("time Windows ", timeWindows)
# # # print()
# # customer = 10
# discreteTimeWindows = []
# for i in timeWindows:
#     discreteTimeWindows.append(discreteTime(i, 10))

# # # print("discreteTimeWindows", discreteTimeWindows)
# # # print()

# # # starting_time = '6:00am'

# # # timeMatrix = [[0, 30, 10], [30, 0, 10], [10, 10, 0]]

# timeMatrix = []
# for i in range(0,customers):
#     timeMatrix.append([0]*customers)
# for i in range(0,customers):
#     for j in range(0,customers):
#         if(i==j):
#             timeMatrix[i][j] = 0
#         elif (i<j):
#             timeMatrix[i][j] = random.randint(1,10)*10
#             timeMatrix[j][i] = timeMatrix[i][j]

# print("timeMatrix", timeMatrix)
# print()



In [5]:
customers = 40
df = pd.read_csv("data_fin.csv")
start = 0
speed = 0.5 # 30km/hr
df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
df.reset_index(inplace = True)


#remove unreachable if(starting time + travelling > end )


print(len(df))
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i+1]-(df['READY TIME'][i+1]%10)
    et = df['DUE DATE'][i+1]-(df['DUE DATE'][i+1]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))



timeMatrix = []
speed = 0.5 # 30km/hr
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            c= int((abs(df['XCOORD.'][i]-df['XCOORD.'][j])+abs(df['YCOORD.'][i]-df['YCOORD.'][j]))/speed)
            if(c%10 == 0 or c%10 == 5):
                timeMatrix[i][j] = c
                timeMatrix[j][i] = timeMatrix[i][j]
            elif(c%10 < 5):
                timeMatrix[i][j] = c-(c%10)
                timeMatrix[j][i] = timeMatrix[i][j]
            elif(c%10 > 5):
                timeMatrix[i][j] = c+(10 - c%10)
                timeMatrix[j][i] = timeMatrix[i][j]                

98


In [6]:


m1 = Model('VRPTW')

count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0, len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()


edgeFlows = {}
edgeNumber = 0
x = [] 
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')
# GRAPH CONSTRUCTION

# out of s to starting of each customer
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#     m1.update()
    edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
    edgeFlows[edgeNumber] = (0, nodeCount[i][0])
    e.append(edgeNumber)
    edgeNumber += 1
edgesList[0] = e

for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])-1):
        e = []
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
#         m1.update()
        edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        # add edge number in edge list
        e.append(edgeNumber)
        edgesList.append(e)
        edgeNumber += 1
    edgesList.append([])
print("Edges List:", edgesList)

    
for i in range(len(discreteTimeWindows)):
    tu = nodeCount[i][-1]
    start_utime = discreteTimeWindows[i][0]
    end_utime = discreteTimeWindows[i][-1]

    # stores edges reachable from u
    for k in range(0, len(discreteTimeWindows)):
        sv = nodeCount[k][0]

        start_vtime = discreteTimeWindows[k][0]
        end_vtime = discreteTimeWindows[k][-1]


        if(i != k and start_utime + timeMatrix[i][k] > end_vtime):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)

              # not reachable

        elif(i != k and start_vtime > end_utime + timeMatrix[i][k]):
            # edge bw (u,tu) -> (v,sv)
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            
            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            edgeFlows[edgeNumber] = (tu, sv)
            print(tu, "->", sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif(i != k and start_vtime == end_utime + timeMatrix[i][k]):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
            

            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            print(tu, "->", sv)
            edgeFlows[edgeNumber] = (tu, sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif (i != k and start_vtime < end_utime + timeMatrix[i][k]):
           
            firstReachableNode_index = 0  # first such that node >= sv
            for j in range(0, len(discreteTimeWindows[i])):
                if(timeMatrix[i][k] + discreteTimeWindows[i][j] >= start_vtime):
                    firstReachableNode_index = j
                    break
            # now starting from firstReachableNode make edges until feasible
            for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                e = []
                reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]
                for r in range(0, len(discreteTimeWindows[k])):
                    if(discreteTimeWindows[k][r] >= reachingTime):
                        print(edgesList[nodeCount[i][l]])
                        print(nodeCount[i][l], "->", nodeCount[k][r])
                        # edge bw these two
#                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
                        
                        edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                        edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                        e.append(edgeNumber)
                        edgesList[nodeCount[i][l]].extend(e)
                        print("hdhdh", edgesList[nodeCount[i][l]])
                        edgeNumber += 1
                        break




print('edge list b4 s t', edgesList)
print()

# into t (except s)
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
    edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
    # add edge number in edge list
    edgesList[nodeCount[i][-1]].append(edgeNumber)
    edgeNumber += 1
edgesList.append([])


print("Node Count after s and t:", nodeCount)
print()

# GRAPH Construction OVER
# if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


offset = edgeNumber

print('edge list', edgesList)
print()
print("edges", edges)
print()
print('edge number', edgeNumber)
print()

# print("edge flows:", edgeFlows)

flows = [0]*customers
for i in range(customers):
    flows[i] = edgeFlows.copy()
    keys = edgeFlows.keys()
    for j in keys:  # stores key values of edgeflows i.e 0 1 2
        # all x[] > offset are for flow variables
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
       
        # remove key and replace it by edge
        t = flows[i].pop(j)
        flows[i][edgeNumber] = t
        edgeNumber += 1

# print("flows", flows)



Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11
Node Count  [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], [31, 32, 33, 34, 35, 36, 37], [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], [52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69], [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], [81, 82, 83, 84, 85, 86, 87, 88, 89], [90, 91, 92, 93, 94, 95, 96, 97, 98], [99, 100, 101, 102, 103, 104, 105, 106, 107, 108], [109, 110, 111, 112, 113, 114, 115], [116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128], [129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141], [142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155], [156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177], [178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191], [192, 193, 194, 195

In [7]:
noc = 0
# # contraint for outgoing flow for a customer
for i in range(0, len(nodeCount)):  # choose one list from nodeCount
#     contraint = [0] * (edgeNumber)
    coeff = [] # store where coeff = 1
    for j in range(0, len(nodeCount[i])):  # se
        node = nodeCount[i][j]
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(v not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1



incomingEdgeList = []
for i in range(0, len(edgesList)):
    incomingEdgeList.append([])  # adding lists in a list
for i in edges:
    v = edges[i][1]  # u -> v

    edge_no = int(i[2:-1])
    incomingEdgeList[v].append(edge_no)

print("incoming edge list: ", incomingEdgeList)
print()

#  constraint for incoming for a customer
for i in range(0, len(nodeCount)):
    coeff = []
    contraint = [0] * (edgeNumber)  # indicates number of edges
    for j in range(0, len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
                # print("u", u, "v", v)
            if(u not in nodeCount[i]):
                    # print('hello')
                    # print("u", u, "v", v)
                    coeff.append(x[e])
    m1.addConstr(( sum(coeff) == 1  ))
    noc+=1
    
# constraint for a node incoming = outgoing
for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])):
        coeff = []
        contraint = [0] * (edgeNumber)
        node = nodeCount[i][j]
        for k in range(0, len(incomingEdgeList[node])):
            e = incomingEdgeList[node][k]
            contraint[e] = 1
            coeff.append(x[e])
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            contraint[e] = -1
            coeff.append(-x[e])
        m1.addConstr(( sum(coeff) == 0 ))
        noc+=1  

incoming edge list:  [[], [0, 549, 713, 800, 985, 1039, 1100, 1145, 1199, 1226, 1347, 1493, 1691, 1774, 1871, 1920, 1989, 2049, 2161, 2211, 2339, 2377, 2546, 2705, 2833, 2904, 3051, 3114, 3204, 3437, 3510, 3678, 3726], [40, 1200, 1990, 2340, 3052], [41, 1201, 1441, 1991, 2341, 3053, 3373], [42, 738, 1442, 1992, 2342, 3054, 3374], [43, 739, 1443, 1993, 2343, 3055, 3375], [44, 740, 1444, 1994, 2344, 3056, 3376], [45, 741, 1445, 3057, 3377], [46, 742, 1446, 3058, 3378], [47, 743, 1447, 3059, 3165, 3379], [48, 744, 1448, 3060, 3166, 3380], [49, 745, 1449, 2307, 3061, 3167, 3381], [50, 746, 1450, 1858, 2308, 3062, 3168, 3382], [51, 747, 1451, 1859, 2309, 3169], [52, 748, 1452, 1860, 2310, 3170], [53, 749, 1453, 1861, 2311, 3171], [1, 801, 1494, 2547, 3205], [54, 802, 1495, 2378, 2548, 3206], [55, 803, 1496, 2379, 2549, 3207], [56, 804, 1497, 2380, 2550, 2905, 3208], [57, 805, 1498, 2381, 2551, 2906, 3209], [58, 806, 1499, 2382, 2552, 2907, 3210, 3511], [59, 807, 1500, 2383, 2553, 2908, 3211

In [8]:
print('done')
# #  0<= xij - fij    f<x => 0<=x-f
for j in range(len(flows)):
    diff = offset*j+offset
    for k in flows[j].keys():
        coeff = [] 
        coeff.append(-x[k])
        coeff.append(x[k-diff])
        m1.addConstr(( sum(coeff) >= 0 ))
        
print('done')
## print(contraint," >= 0")

# constraint for a particular customer outgoing == 0 for c the customer for rest dont care
for c in range(customers):
    addn = offset*(c+1)
    
    for i in range(0, len(nodeCount)):
        if(i == c):
        # indicates number of edges
            coeff = []
            for j in range(0, len(nodeCount[i])):
                node = nodeCount[i][j]
                for k in range(0, len(edgesList[node])):
                    e = edgesList[node][k]
                    u = edges['x[%i]' % e][0]
                    v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                    if(v not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        edge = 'x[%i]' % e
                        node = edges[edge][0]
    #                     print("node:",node)
                        coeff.append(x[e+addn])
            m1.addConstr(( sum(coeff) == 0 ))
                    

print('done w edges')        






print("CONSTRAINT for incoming = 1 ")
for c in range(customers):
    addn = offset*(c+1)
    
    for i in range(0, len(nodeCount)):
        # indicates number of edges
        coeff = []
        if( i == c):
            for j in range(0, len(nodeCount[i])):
                node = nodeCount[i][j]
                for k in range(0, len(incomingEdgeList[node])):
                    e = incomingEdgeList[node][k]
                    u = edges['x[%i]' % e][0]
                    v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                    if(u not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e+addn])
            m1.addConstr(( sum(coeff) == 1 ))
                    
                    


# # flow conservation  for every node except those present in customer = c

for c in range(customers):
    addn = offset*c+offset
    for i in range(0, len(nodeCount)):
        if ( i != c):
            for j in range(0, len(nodeCount[i])):
                coeff = []
                node = nodeCount[i][j]
                for k in range(0, len(incomingEdgeList[node])):
                    e = incomingEdgeList[node][k]
                    coeff.append(x[e+addn])
                for k in range(0, len(edgesList[node])):
                    e = edgesList[node][k]
                    coeff.append(-x[e+addn])
                m1.addConstr(( sum(coeff) == 0 ))

done
done
done w edges
CONSTRAINT for incoming = 1 


In [9]:
m1.display()

Minimize
  <gurobi.LinExpr: 0.0>
Subject To


In [10]:
objectivefun = []
for i in range(0, len(edgesList[0])):
    objectivefun.append(x[edgesList[0][i]])
obj = sum(objectivefun)
m1.setObjective(obj,GRB.MINIMIZE)
  
# m1.Params.MIPGapAbs= 5 
 
# m1.Params.TimeLimit = 300  # 5 minutes
m1.params.Method = 2 # fix to barrier
m1.optimize(mycallback)
print("Runtime is :",m1.Runtime)
print("objective value : ",m1.objVal)

Set parameter Method to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 173240 rows, 156784 columns and 621924 nonzeros
Model fingerprint: 0x71ab4c0f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 5369 rows and 5288 columns
Presolve time: 0.64s
Presolved: 167871 rows, 151496 columns, 609770 nonzeros

Elapsed ordering time = 5s
Elapsed ordering time = 7s
Elapsed ordering time = 10s
Elapsed ordering time = 10s
Elapsed ordering time = 14s
Elapsed ordering time = 15s
Ordering time: 17.58s

Barrier statistics:
 AA' NZ     : 3.725e+06
 Factor NZ  : 9.455e+07 (roughly 900 MB of memory)
 Factor Ops : 3.407e+11 (roughly 6 seconds per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Prim

In [11]:
for v in m1.getVars():
    print('%s: %g' % (v.varName,v.x))

AttributeError: Unable to retrieve attribute 'x'

In [ ]:
print("Number of Constraints :",noc)
print("Runtime is :",m1.Runtime)

In [ ]:
ans = {}
for v in m1.getVars()[:offset]:
    if (v.x > 0):
#         print(v.varName[2:-1])
        ans[(v.varName[2:-1])]=v.x
ans

In [ ]:
edges
new_edges = {}   # stores only those edges which are required
for key in edges:
    if((key[2:-1]) in ans):
        new_edges[(key[2:-1])] = edges[key]
new_edges

In [ ]:
graphForpaths = []
for i in range(len(edgesList)): # edgesList contains outgoing from each node
    l = []
    for key in new_edges:
        if(new_edges[key][0] == i):
            if(new_edges[key][1] == -1):
                l.append(len(edgesList)-1)
            else:
                l.append(new_edges[key][1])
    graphForpaths.append(l)
graphForpaths

In [ ]:
len(graphForpaths)

In [ ]:
def dfs(v,visited,stack,graph):
        # Mark the current node as visited.
        visited[v] = True
        for i in graph[v]:
            if visited[i] == False:
                dfs(i,visited,stack,graph)
        stack.insert(0,v)
def topologicalSort(visited,graph):
        stack =[] 
        dfs(0,visited,stack,graph)
        print (stack)
        return stack

visited = [False]*len(graphForpaths)
ts = topologicalSort(visited,graphForpaths)

In [ ]:
def length(u,v,nodeCount):
    for i in range(len(nodeCount)):
        if(u in nodeCount[i]):
            if(v in nodeCount[i]):
                return 0
            else:
                return 1
        
        if(v in nodeCount[i]):
            if(u in nodeCount[i]):
                return 0
            else:
                return 1
            

# dp[i] stores maximum length from s to i
path = [0]*len(ts)              # stores index of path for which max occurs at i
dp = [0]*len(ts)
dp[0] = 1
for i in range(1,len(ts)):
    u = ts[i]
    for j in range(0,i):
        v = ts[j]
        if(u in graphForpaths[v]):
            if(length(u,v,nodeCount) + dp[j] > dp[i]):
                dp[i] = length(u,v,nodeCount) + dp[j]
                path[i] = j
#             dp[i] = max(dp[i],length(u,v,nodeCount) + dp[j])
        
print("max length :",dp[-1]-2)
print(path)
route = []
route.append(-1)
curr = -1
while(True):
    if(curr == 0):
        break
    prev = path[curr]
    route.append(ts[prev])
    curr = prev
    
route.reverse()
print("longest route:",route)
    
    
# now i need longest 
customer,visited = distinctCustomer(route,nodeCount)
visited.sort(reverse = True)
print(customer)
print(visited)  

In [ ]:
customer,visited = distinctCustomer(route,nodeCount)
visited.sort(reverse = True)
print(customer)
print(visited)

In [ ]:
for i in range(len(paths)):
    paths[i][-1] = -1
paths

In [ ]:
nodeCount

In [ ]:
def distinctCustomer(path,nodeCount):
    customer = 0
    visited = [] # stores index of visited customer
    for i in range(1,len(path)-1):
        node = path[i]
        for j in range(0,len(nodeCount)):
            if(node in nodeCount[j]):
                if(j not in visited):
                    visited.append(j)
                    customer+=1
    return customer,visited

In [ ]:
def maxCustomerpath(paths,nodeCount):
    maxPaths = 0
    maxCustomer = 0
    for i in range(0,len(paths)):
        path = paths[i]
        customer,visited = distinctCustomer(path,nodeCount)
        if(maxCustomer < customer):
            maxCustomer = customer
            maxPaths = visited
            
    return maxCustomer,maxPaths
        
        

distict_customers,customer_nodes =  maxCustomerpath(paths,nodeCount)      
distict_customers

In [ ]:
customer_nodes.sort(reverse = True)
customer_nodes

In [ ]:
timeWindows = [[0, 30], [1, 30], [2, 20], [3, 40], [4, 40], [5, 50], [6, 30], [7, 30], [8, 30], [9, 30]]
# discreteTimeWindows [[20, 30], [20, 30], [10, 20], [20, 30, 40], [20, 30, 40], [20, 30, 40, 50], [10, 20, 30], [10, 20, 30], [20, 30], [20, 30]]
# timeMatrix =  [[0, 40, 70, 20, 90, 90, 60, 80, 40, 30], [40, 0, 10, 40, 40, 70, 50, 30, 30, 60], [70, 10, 0, 80, 100, 60, 50, 50, 70, 100], [20, 40, 80, 0, 80, 70, 40, 10, 70, 80], [90, 40, 100, 80, 0, 100, 10, 100, 40, 40], [90, 70, 60, 70, 100, 0, 10, 20, 20, 90], [60, 50, 50, 40, 10, 10, 0, 40, 80, 10], [80, 30, 50, 10, 100, 20, 40, 0, 100, 80], [40, 30, 70, 70, 40, 20, 80, 100, 0, 60], [30, 60, 100, 80, 40, 90, 10, 80, 60, 0]]

In [ ]:
#modify time-windows
for i in range(0,len(customer_nodes)):
    del timeWindows[customer_nodes[i]]
timeWindows


In [ ]:
discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))
discreteTimeWindows

In [ ]:
timeMatrix =  [[0, 1, 2, 3, 4,5, 6, 7, 8, 9], [40, 0, 10, 40, 40, 70, 50, 30, 30, 60], [70, 10, 0, 80, 100, 60, 50, 50, 70, 100], [20, 40, 80, 0, 80, 70, 40, 10, 70, 80], [90, 40, 100, 80, 0, 100, 10, 100, 40, 40], [90, 70, 60, 70, 100, 0, 10, 20, 20, 90], [60, 50, 50, 40, 10, 10, 0, 40, 80, 10], [80, 30, 50, 10, 100, 20, 40, 0, 100, 80], [40, 30, 70, 70, 40, 20, 80, 100, 0, 60], [30, 60, 100, 80, 40, 90, 10, 80, 60, 0]]

In [ ]:
#modify time-Matrix
# timeMatrix = [[1,2,3],[2,1,3],[3,2,1]]
# l=[2]
for row in customer_nodes:
    del timeMatrix[row]
    
for row in range(0,len(timeMatrix)):
    for col in customer_nodes:
        del timeMatrix[row][col]
    
    
    
timeMatrix

In [ ]:
l=[]
len(l)

In [ ]:
import pandas as pd
df= pd.read_csv("data_fin.csv")


In [ ]:
timeWindows = [] 
for i in range(0,100):
    st = df['READY TIME'][i]-(df['READY TIME'][i]%10)
    et = df['DUE DATE'][i]-(df['DUE DATE'][i]%10)
    tw = [st,et]
    timeWindows.append(tw)
    
timeWindows

In [ ]:
# discreteTimeWindows = []
# for i in timeWindows:
#     discreteTimeWindows.append(discreteTime(i,10))
# discreteTimeWindows

In [ ]:
# timeMatrix = []
# speed = 0.5 # 30km/hr
# for i in range(0,50):
#     timeMatrix.append([0]*50)
# for i in range(0,50):
#     for j in range(0,50):
#         if(i==j):
#             timeMatrix[i][j] = 0
#         elif (i<j):
#             c= int(abs(df['XCOORD.'][i]-df['XCOORD.'][j])+abs(df['YCOORD.'][i]-df['YCOORD.'][j]))/speed
#             if(c%10 == 0 or c%10 == 5):
#                 timeMatrix[i][j] = c
#                 timeMatrix[j][i] = timeMatrix[i][j]
#             elif(c%10 < 5):
#                 timeMatrix[i][j] = c-(c%10)
#                 timeMatrix[j][i] = timeMatrix[i][j]
#             elif(c%10 > 5):
#                 timeMatrix[i][j] = c+(10 - c%10)
#                 timeMatrix[j][i] = timeMatrix[i][j]
                
# timeMatrix

In [ ]:
# df1 = df[ df['DUE DATE'] < (df['XCOORD.']+df['YCOORD.'])/speed ]
# # df

# 